In [1]:
+import json
import pandas as pd

In [2]:
questions_train = pd.read_csv('../data/train.tsv', delimiter='\t')
questions_dev = pd.read_csv('../data/dev.tsv', delimiter='\t')

In [45]:
questions_train

,dataset,example_uid,question,answer,turker_answer,rule-based
0,SQuAD,572812523acd2414000df3bf,What was Prince Albert 's nickname ?,Bertie,Prince Albert 's nickname was Bertie .,Prince Albert 's nickname was Bertie .
1,SQuAD,572817584b864d1900164464,Where is the centre of London said to be locat...,Eleanor Cross at Charing Cross near the juncti...,The centre of London is said to be located by ...,The centre of London is said to be located in ...
2,SQuAD,57312c6005b4da19006bce36,What pigment was used to paint villas in ancie...,vermilion,Vermillion pigment was used to paint villas in...,Vermilion was used to paint villas in ancient ...
3,SQuAD,57266b70708984140094c56a,Who was the Queen 's first grandchild ?,Wilhelm,Wilhelm was the Queen 's first grandchild .,Wilhelm was the Queen 's first grandchild .
4,SQuAD,56db66a2e7c41114004b509a,Who did BMG partner with in 2005-2007 ?,Sony Music Entertainment,BMG partnered with Sony Music Entertainment in...,BMG partner with Sony Music Entertainment in 2...
...,...,...,...,...,...,...
60705,SQuAD,572fddffb2c2fd1400568532,Who operated the automatic beam system ?,the bomber 's autopilot,The bomber 's autopilot operated the automatic...,The bomber 's autopilot operated the automatic...
60706,SQuAD,572e94eecb0c0d14000f1308,What two mountains ranges are located on the i...,Troodos Mountains and the smaller Kyrenia Range,The Troodos Mountains and the smaller Kyrenia ...,Troodos Mountains and the smaller Kyrenia Rang...
60707,SQuAD,573117cae6313a140071cc37,How do 15 % of male Tuvaluans earn their income ?,seamen,15 % of male Tuvaluans earn their income by se...,15 % of male Tuvaluans earn their income by se...
60708,SQuAD,570d1ef7fed7b91900d45c2c,Which Apple campaign did some people feel enco...,"`` Rip , Mix , Burn ''","Some people felt Apple 's `` Rip , Mix , Burn ...","`` Rip , Mix , Burn did some people feel encou..."


In [3]:
squad2_train = json.load(open('../data/train-v2.0.json'))
squad2_dev = json.load(open('../data/dev-v2.0.json'))

In [4]:
squad2_train['data'][0]['paragraphs'][0]['qas']

[{'question': 'When did Beyonce start becoming popular?',
  'id': '56be85543aeaaa14008c9063',
  'answers': [{'text': 'in the late 1990s', 'answer_start': 269}],
  'is_impossible': False},
 {'question': 'What areas did Beyonce compete in when she was growing up?',
  'id': '56be85543aeaaa14008c9065',
  'answers': [{'text': 'singing and dancing', 'answer_start': 207}],
  'is_impossible': False},
 {'question': "When did Beyonce leave Destiny's Child and become a solo singer?",
  'id': '56be85543aeaaa14008c9066',
  'answers': [{'text': '2003', 'answer_start': 526}],
  'is_impossible': False},
 {'question': 'In what city and state did Beyonce  grow up? ',
  'id': '56bf6b0f3aeaaa14008c9601',
  'answers': [{'text': 'Houston, Texas', 'answer_start': 166}],
  'is_impossible': False},
 {'question': 'In which decade did Beyonce become famous?',
  'id': '56bf6b0f3aeaaa14008c9602',
  'answers': [{'text': 'late 1990s', 'answer_start': 276}],
  'is_impossible': False},
 {'question': 'In what R&B group

In [73]:
num_past_questions = 4

all_ids_dict = {}
for item in squad2_train['data']:
    for paragraph in item['paragraphs']:
        questions = [qa['question'] for qa in paragraph['qas']]
        answers = [qa['answers'][0]['text'] if qa['answers'] else 'unknown' for qa in paragraph['qas']]
        for index, qa in enumerate(paragraph['qas']):
            text = '\n'.join(['Q: ' + q
                             + '\nA: ' + a 
                             for q, a in zip(questions[max(0, index-num_past_questions):index+1], 
                                             answers[max(0, index-num_past_questions):index+1]) 
                  ])
            all_ids_dict[qa['id']] = text

In [61]:
all_ids = set()
for item in squad2_train['data'] + squad2_dev['data']:
    for paragraph in item['paragraphs']:
        for qa in paragraph['qas']:
            all_ids.add(qa['id'])

In [62]:
len(all_ids_dict)

130319

In [63]:
len(all_ids)

142192

In [64]:
all_ids_dict[list(all_ids)[1]]

'Q: What has been an important atmospheric study since the 1970s?\nA: ozone layer'

In [92]:
def clean_line(line):
    line = line.replace('-LRB-', '(')
    line = line.replace('-RRB-', ')')
    line = line.replace('-LSB-', '[')
    line = line.replace('-RSB-', ']')
    line = line.replace('( ', '(')
    line = line.replace(' )', ')')
    line = line.replace('[ ', '[')
    line = line.replace(' ]', ']')
    line = line.replace(' .', '.')
    line = line.replace(' , ', ', ')
    line = line.replace(' ; ', '; ')
    line = line.replace(' : ', ': ')
    return line

In [93]:
def get_text_list(data):
    all_texts = []
    for index, row in data.iterrows():
        qid, answer = row['example_uid'], row['turker_answer']
        answer = answer[0].upper() + answer[1:]
        answer = clean_line(answer)
        if qid not in all_ids_dict:
            continue
        text = 'Discussion:\n'
        text += all_ids_dict[qid]
        text += '\n\n'
        text += 'Statement:\n'
        text += answer
        all_texts.append(text)
        
    return all_texts

In [94]:
train_texts_list = get_text_list(questions_train)
dev_texts_list = get_text_list(questions_dev)

In [95]:
with open('../data/question_to_statement_train.json', 'w') as f:
    json.dump(train_texts_list, f)
with open('../data/question_to_statement_dev.json', 'w') as f:
    json.dump(dev_texts_list, f)

In [98]:
print(dev_texts_list[3])

Discussion:
Q: After the war, what did Napoleon focus on?
A: domestic affairs
Q:  When did Napoleon divorce Josephine?
A: January 1810
Q: After his divorce from Josephine, who did Napoleon marry?
A: Archduchess Marie Louise

Statement:
Napoleon married Archduchess Marie Louise after his divorce from Josephine.
